In [48]:
import pandas as pd
import numpy as np

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
path = "/content/drive/My Drive/Hackhathon 2020 Shared Drive/Dataset/SG_car_cleaned.csv"

In [51]:
data = pd.read_csv(path)

In [52]:
data = data.drop(['Unnamed: 0','Unnamed: 0.1', 'trj_id', 'osname', 'pingtimestamp', 'timestamp', 'latitude', 'longitude', 'speed', 'bearing', 'accuracy', 'pickup_time', 'dropoff_time'], axis=1)

In [53]:
data

,travel_time,latitude_origin,longitude_origin,latitude_destination,longitude_destination,hour_of_day,day_of_week,is_weekend,is_weekday,is_wee_hours,is_rush_hours_morning,is_rush_hours_evening,sin_hour_of_day,cos_hour_of_day,sin_day_of_week,cos_day_of_week,haversine_km,haversine_m,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1
0,1520.0,1.270377,103.857875,1.446609,103.806354,2,1,0,1,0,0,0,5.000000e-01,0.866025,0.781831,0.623490,20.403093,12.677028,-0.025486,-0.076384,0.041837,0.094437
1,1158.0,1.282591,103.851082,1.332855,103.977278,8,4,0,1,0,0,0,8.660254e-01,-0.500000,-0.433884,-0.900969,15.091547,9.376812,-0.017611,-0.064838,-0.138717,-0.003314
2,1366.0,1.355363,103.863157,1.343218,103.858504,10,5,1,0,0,0,1,5.000000e-01,-0.866025,-0.974928,-0.222521,1.445192,0.897939,-0.023023,0.008731,-0.019494,-0.003787
3,1328.0,1.340301,103.775742,1.303071,103.900220,1,5,1,0,0,0,0,2.588190e-01,0.965926,-0.974928,-0.222521,14.434456,8.968542,0.062661,-0.014213,-0.064685,-0.039977
4,1210.0,1.333936,103.953225,1.309601,103.833983,7,3,0,1,0,0,0,9.659258e-01,-0.258819,0.433884,-0.900969,13.520439,8.400637,-0.114666,-0.004423,0.001871,-0.039493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27537,1112.0,1.396872,103.747718,1.319548,103.843496,22,0,0,1,0,0,0,-5.000000e-01,0.866025,0.000000,1.000000,13.676692,8.497722,0.095710,0.039577,-0.006699,-0.028723
27538,983.0,1.396533,103.747537,1.322236,103.841737,22,1,0,1,0,0,0,-5.000000e-01,0.866025,0.781831,0.623490,13.329784,8.282178,0.095859,0.039224,-0.004703,-0.026205
27539,1382.0,1.301317,103.899443,1.410657,103.778843,1,6,1,0,0,0,0,2.588190e-01,0.965926,-0.781831,0.623490,18.086938,11.237934,-0.064071,-0.041795,0.065966,0.056133
27540,1212.0,1.298612,103.853801,1.341311,103.981728,11,2,0,1,0,0,1,2.588190e-01,-0.965926,0.974928,-0.222521,14.983214,9.309501,-0.018864,-0.048636,-0.142380,0.005511


# Model Check Script

In [ ]:
def xgboost(X_train, Y_train, X_val, Y_val, X_test, Y_test):
  dtrain = xgb.DMatrix(X_train, label=Y_train)
  dvalid = xgb.DMatrix(X_val, label=Y_val)
  dtest = xgb.DMatrix(X_test)
  watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

  model1 = xgb.XGBRegressor(n_estimators = 4000, min_child_weight= 10,
                          learning_rate=0.005, max_depth=10,
                          colsample_bytree= 0.9,subsample= 0.9, nthread=-1, 
                          booster= 'gbtree', silent= 1,eval_metric= 'rmse', 
                          objective= 'reg:linear',early_stopping_rounds=40, 
                          maximize = False, verbose_eval=1)
  model1.fit(X_train, Y_train)
  y_pred1 = model1.predict(X_test)
  mse = mean_squared_error(Y_test,y_pred1)
  return np.sqrt(mse)

XGBoost only Modeller Script

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import xgboost as xgb

def final_modeller(n_modelling,data):  
  result_matrix = [0]*n_modelling

  for i in range(n_modelling):
    data = data.sample(frac=1).reset_index(drop=True)
    train, test = train_test_split(data, test_size=0.3, train_size = 0.7, random_state = 11)
    validate, test = train_test_split(test, test_size = 0.3, train_size = 0.7, random_state = 11)

    train_scaled = preprocessing.scale(train[train.columns.difference(['travel_time'])])
    X_train = train_scaled
    Y_train = train["travel_time"]
    val_scaled = preprocessing.scale(validate[validate.columns.difference(['travel_time'])])
    X_val = val_scaled
    Y_val = validate["travel_time"]
    test_scaled = preprocessing.scale(test[test.columns.difference(['travel_time'])])
    X_test = test_scaled
    Y_test = test["travel_time"]
    xgbResult = xgboost(X_train, Y_train, X_val, Y_val, X_test, Y_test)
    result_matrix[i] = xgbResult
    print("Iteration "+str(i)+" = " + str(xgbResult))

  xgbAvg, xgbMin, xgbMax = 0, 1000, 0

  for i in range(n_modelling):
    xgbAvg += result_matrix[i]
    if xgbMin > result_matrix[i]:
      xgbMin = result_matrix[i]
    if xgbMax < result_matrix[i]:
      xgbMax = result_matrix[i]

  xgbAvg = xgbAvg/n_modelling

  print("The mean loss statistics for each model:")
  print("xgboost            : Max="+str(round(xgbMax,2))+"; Min="+str(round(xgbMin,2))+"; Avg="+str(round(xgbAvg)))
  from google.colab import output
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
  return result_matrix

In [ ]:
print(final_modeller(10, data))

Iteration 10 = 200.0211390321384
Iteration 10 = 197.02434573512394
Iteration 10 = 198.71902529362916
Iteration 10 = 194.15387007297826
Iteration 10 = 200.0210170177251
Iteration 10 = 202.57387003697156
Iteration 10 = 192.2840865719062
Iteration 10 = 198.78897679902366
Iteration 10 = 194.20779586415205
Iteration 10 = 203.6008435447858
The mean loss statistics for each model:
xgboost            : Max=203.6; Min=192.28; Avg=198.0
[200.0211390321384, 197.02434573512394, 198.71902529362916, 194.15387007297826, 200.0210170177251, 202.57387003697156, 192.2840865719062, 198.78897679902366, 194.20779586415205, 203.6008435447858]


## MODELLING

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

data = data.sample(frac=1).reset_index(drop=True)
train, test = train_test_split(data, test_size=0.3, train_size = 0.7, random_state = 11)
validate, test = train_test_split(test, test_size = 0.3, train_size = 0.7, random_state = 11)
print(len(train))
print(len(validate))
print(len(test))

from sklearn import preprocessing
train_scaled = preprocessing.scale(train[train.columns.difference(['travel_time'])])
X_train = train_scaled
Y_train = train["travel_time"]
val_scaled = preprocessing.scale(validate[validate.columns.difference(['travel_time'])])
X_val = val_scaled
Y_val = validate["travel_time"]
test_scaled = preprocessing.scale(test[test.columns.difference(['travel_time'])])
X_test = test_scaled
Y_test = test["travel_time"]

19279
5784
2479


In [55]:
import timeit
"""start = timeit.default_timer()
main()
end = timeit.default_timer()
print("Time taken in seconds:{}".format(end-start))"""

'start = timeit.default_timer()\nmain()\nend = timeit.default_timer()\nprint("Time taken in seconds:{}".format(end-start))'

XGBoost

In [57]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=Y_train)
dvalid = xgb.DMatrix(X_val, label=Y_val)
dtest = xgb.DMatrix(X_test)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

In [58]:
model1 = xgb.XGBRegressor(n_estimators = 4000, min_child_weight= 10,
                          learning_rate=0.005, max_depth=10,
                          colsample_bytree= 0.9,subsample= 0.9, nthread=-1, 
                          booster= 'gbtree', silent= 1,eval_metric= 'rmse', 
                          objective= 'reg:linear',early_stopping_rounds=40, 
                          maximize = False, verbose_eval=1)

In [59]:
model1.fit(X_train, Y_train)
y_pred1 = model1.predict(X_test)
mse = mean_squared_error(Y_test,y_pred1)
print("RMSE: %.2f" % np.sqrt(mse))

RMSE: 194.74


In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 2, figsize=[15, 4])

# Plot the log loss during training
axs[0].plot(evaluation_results['Train']['rmse'], label='Train')
axs[0].plot(evaluation_results['Valid']['rmse'], label='Valid')
axs[0].set_ylabel('RMSE')
axs[0].set_xlabel('Boosting round')
axs[0].set_title('Training performance')
axs[0].legend()

plt.show()

LightGBM

In [ ]:
import lightgbm as lgb

d_train = lgb.Dataset(X_train, label=Y_train)
d_valid = lgb.Dataset(X_val, label=Y_val)

In [64]:
model2 = lgb.LGBMRegressor(learning_rate=0.005,boosting_type="gbdt",
                           sub_feature=0.9,objective = "regression",
                           num_leaves=20,min_data = 22,max_depth=10,max_bin=10,
                           n_estimators = 4000)

In [65]:
model2.fit(X_train, Y_train)
y_pred2 = model2.predict(X_test)
mse = mean_squared_error(Y_test,y_pred2)
print("RMSE: %.2f" % np.sqrt(mse))

RMSE: 202.84


Stacking XGBoost & LightGBM

In [ ]:
from sklearn.ensemble import VotingRegressor

In [71]:
estimators = [("xgb", model1), ("lgbm", model2)]

model = VotingRegressor(estimators=estimators)

In [72]:
model.fit(X_train, Y_train)

VotingRegressor(estimators=[('xgb',
                             XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=0.9,
                                          early_stopping_rounds=40,
                                          eval_metric='rmse', gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.005, max_delta_step=0,
                                          max_depth=10, maximize=False,
                                          min_child_weight=10, missing=None,
                                          n_estimators=4000, n_jobs=1,
                                          nthread=-1...
                                           importance_type='split',
                                           learning_rate=0.005, max_b

In [73]:
y_pred = model.predict(X_test)
mse = mean_squared_error(Y_test,y_pred)
print("RMSE: %.2f" % np.sqrt(mse))

RMSE: 195.72


In [74]:
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

Iteration 1 = 200.0211390321384

Iteration 2 = 197.02434573512394

Iteration 3 = 198.71902529362916

Iteration 4 = 194.15387007297826

Iteration 5 = 200.0210170177251

Iteration 6 = 202.57387003697156

Iteration 7 = 192.2840865719062

Iteration 8 = 198.78897679902366

Iteration 9 = 194.20779586415205

Iteration 10 = 203.6008435447858

The mean loss statistics for each model:

xgboost            : Max=203.6; Min=192.28; Avg=198.0
[200.0211390321384, 197.02434573512394, 198.71902529362916, 194.15387007297826, 200.0210170177251, 202.57387003697156, 192.2840865719062, 198.78897679902366, 194.20779586415205, 203.6008435447858]